In [1]:
SAMPLE = 2  # pick sample from the dev set

# Predict

In [2]:
# match entity label to the ctc table
import os
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from datasets import load_dataset
import soundfile as sf

CTC_DEPTH = 3  # size of the ctc matrix considered for search
NPATHS = 10 # number of longest paths on the bigram graph

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")


path = "../data/dev/"

file = str(SAMPLE) + '.wav'
speech, samplerate = sf.read(path+file)
i = int(file.split('.')[0]) - 1

input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values
logits = model(input_values).logits

# find where s_tokens appear in the table
ctc_table = torch.topk(logits, k=CTC_DEPTH, dim=-1)
predicted_ids = ctc_table.indices[0]
# predicted_ids = torch.argmax(logits, dim=-1).indices

print(predicted_ids)
print(predicted_ids.shape)

# greedy decoding
transcription = tokenizer.batch_decode(torch.argmax(logits, dim=-1))[0].lower()
print(transcription)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


tensor([[ 0,  6, 10],
        [ 0,  6, 10],
        [ 0,  6, 10],
        ...,
        [ 0,  6,  4],
        [ 0,  6,  4],
        [ 0,  6,  4]])
torch.Size([365, 3])
what is a hume directed by vipcafon caronsfelt


In [3]:
import numpy as np

predictions = np.transpose(np.array(predicted_ids))
# print(predictions)
# print(predictions.shape)
# print(predictions[0])
indices = predictions.flatten()
# print(indices)

In [4]:
predictions_logits = np.transpose(ctc_table.values[0].detach().numpy())
# print(predictions)
print(predictions_logits.shape)
# print(all_logits[0])
indices_logits = predictions_logits.flatten()
print(indices_logits)

(3, 365)
[15.416867  15.432795  15.467757  ... -2.065003  -1.8960949 -1.6134164]


In [5]:
# predictions = np.array([[2, 0, 0, 4, 5, 5],
#                         [4, 5, 5, 6, 6, 1]])
# generate adjacencies
def connect(predictions, t, k, n):
    edges = []
    for j in range(predictions.shape[0]):  # offset
        if predictions[j][k] != 0:
            if predictions_logits[j][k] > 0:
                edges.append([n*predictions.shape[1]+t, j*predictions.shape[1]+k])
        else:
            # skip to next if exists
            if k < predictions.shape[1]-1:
                edges.extend(connect(predictions, t, k+1, n))
    return edges

edges = []
for t in range(predictions.shape[1]-1):  # columns
    for n in range(predictions.shape[0]):  # rows
        if predictions[n][t] != 0:
            edges.extend(connect(predictions, t, t+1, n))

print(len(edges))

6107


In [6]:
# TODO represent speech as bigrams with probs
from collections import defaultdict

special_tokens = [4]
vocabulary = tokenizer.tokenizer.convert_ids_to_tokens

bigrams = defaultdict(int)
for e in edges:
    i1 = indices[e[0]]
    i2 = indices[e[1]]
    # skip repeated predictions of the same letters (not a bigram)
    if i1 != i2 and i1 not in special_tokens and i2 not in special_tokens:
        bigrams[''.join(vocabulary([i1, i2]))] += indices_logits[e[0]] + indices_logits[e[1]]
print(len(bigrams))

print([k for k, v in sorted(bigrams.items(), key=lambda x: x[1], reverse=True)][:10])

357
['TW', 'TH', 'TA', 'TE', 'IW', 'IH', 'IA', 'ED', 'IE', 'TI']


# Bottom up

In [7]:
# load entities
import json

path = '../data/'

with open(path+'entities.json', 'r') as fin:
    entities = json.load(fin)
print(len(entities), 'entity labels')

28497 entity labels


In [8]:
# load original question
import re
from unidecode import unidecode
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

path = '../data/'
with open(path+'annotated_wd_data_valid_answerable.txt') as fin:
    lines = fin.readlines()
    l = lines[SAMPLE-1]
#         subject [tab] property [tab] object [tab] question
    s, p, o, q = l.strip('\n').split('\t')
    
    q = re.sub(chars_to_ignore_regex, '', q).lower()
    q = unidecode(q)
    q = ''.join([j for i, j in enumerate(q) if j != q[i-1]])  # remove repeated letters
    print(q)

    s_label = entities[s]
    s_label = re.sub(chars_to_ignore_regex, '', s_label).lower()
    s_label = unidecode(s_label)
    s_label = ''.join([j for i, j in enumerate(s_label) if j != s_label[i-1]])  # remove repeated letters
    print(s_label)

what is a film directed by wiebke von carolsfeld
wiebke carolsfeld


In [9]:
# # generate all bigrams
from collections import Counter

# get all bigrams
bigrams = []
for e in edges:
    i1 = indices[e[0]]
    i2 = indices[e[1]]
    # skip repeated predictions of the same letters (not a bigram)
    if i1 != i2 and i1 not in special_tokens and i2 not in special_tokens:
        bigrams.append(''.join(vocabulary([i1, i2])).lower())  # normalise to lower case
# count all bigrams frequency in ctc matrix
bigrams = Counter(bigrams)
        

# special_tokens = [4]

# vocabulary = tokenizer.tokenizer.convert_ids_to_tokens
# def connect(predictions, t, k, n):
#     edges = []
#     for j in range(predictions.shape[0]):  # offset
#         if predictions[j][k] != 0:# and predictions[j][k] != predictions[n][t]:
#             # skip repeated predictions of the same letters (not a bigram)
#             if predictions[n][t] != predictions[j][k]:
#                 if predictions[n][t] not in special_tokens and predictions[j][k] not in special_tokens:
#                     edges.append(''.join(vocabulary([predictions[n][t], predictions[j][k]])))
#         else:
# #             predictions[j][k] = 0
#             # skip to next if exists
#             if k < predictions.shape[1]-1:
#                 edges.extend(connect(predictions, t, k+1, n))
#     return edges

# edges = []
# for t in range(predictions.shape[1]-1):  # columns
#     for n in range(predictions.shape[0]):  # rows
#         if predictions[n][t] != 0:
#             edges.extend(connect(predictions, t, t+1, n))

# bigrams = Counter(edges)
# print(len(edges), 'edges')
# print(len(bigrams), 'bigrams')

In [10]:
# load all entity labels that match the top bigrams
import json

path = '../data/'

with open(path+'bigrams_index.json', 'r') as fin:
    entity_bigrams = json.load(fin)
print(len(entity_bigrams), 'entity bigrams')

1000 entity bigrams


In [11]:
# #
# e_labels = Counter()
# for b, c in bigrams.most_common(10):
#     e_ids = entity_bigrams[b.lower()]
#     for e_id in e_ids:
#         e_label = list(entities.values())[e_id]
#         e_labels[e_label] += 1
# print(len(e_labels))

# top_labels = [l for l, c in e_labels.items() if c > 1]
# print(len(top_labels))

In [12]:
# bottom up from greedy decoding
bottom_bigrams = set([word[i:i + 2] for word in transcription.lower().split() for i in range(0, len(word)-1, 1)])

# for query in s_label.lower().split():
#     print(query)
#     for i in range(0, len(query)-1, 1):
#         bigram = query[i:i + 2]
#         if bigram in bottom_bigrams:
#             print(bigram)
            
# match to the bigrams from the greedy decoded transcription
e_labels = Counter()
for b in bottom_bigrams:
    matched_e_labels = entity_bigrams[b]
    for e_label in matched_e_labels:
        e_labels[e_label] += 1
        if e_label == 'cairo':
            print(b)
print(len(e_labels))

ir
ca
ro
21089


In [13]:
# check entity matches top bigrams
# top_bigrams = [k.lower() for k, v in sorted(bigrams.items(), key=lambda x: x[1], reverse=True)][:20]
# [b for b, c in bigrams.most_common(10)]
# print(top_bigrams)

for query in s_label.lower().split():
    print(query)
    for i in range(0, len(query)-1, 1):
        bigram = query[i:i + 2]
        if bigram in bottom_bigrams:
            print(bigram)

wiebke
carolsfeld
ca
ar
ro
sf
fe
el


In [14]:
topn = 500

print(bottom_bigrams)
# print(e_labels.most_common(topn))


top_labels = [l for l, c in e_labels.most_common(topn)]
print(len(top_labels))

print(s_label in top_labels)

{'ir', 'by', 'te', 'at', 'ip', 'ha', 'sf', 'me', 'el', 'hu', 'lt', 'pc', 'ed', 'on', 'fe', 'fo', 'ns', 'um', 'ca', 'di', 'ro', 'af', 're', 'ct', 'is', 'vi', 'ar', 'wh', 'ec'}
500
True


# Top down

In [15]:
import networkx as nx
import itertools
import difflib


def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return pos_a, pos_b, size


def match(edges, indices, query_str, tokenizer, n_paths=NPATHS):
    query = tokenizer.tokenizer(query_str)['input_ids']
    query = [query[i:i + 2] for i in range(0, len(query)-1, 1)]
    
    # filter bigrams
    bigrams = []
    for e in edges:
        bigram = [indices[e[0]], indices[e[1]]]
        if bigram in query:
            bigrams.append(e)

    # build graph
    DG = nx.DiGraph()
    DG.add_edges_from(bigrams)
    
    # find all paths
#     all_paths = []
#     for (x, y) in itertools.combinations(DG.nodes, 2):
#         for path in nx.all_simple_paths(DG, x, y):
#             all_paths.append(path)
#     # sort all paths
#     all_paths.sort(key=len, reverse=True)
    
    all_paths = [nx.dag_longest_path(DG)]

    # lookup maximum overlap between strings
    for path in all_paths[:n_paths]:
        word = ''.join(tokenizer.tokenizer.convert_ids_to_tokens([indices[i] for i in path]))
#         print(word)
        pos_a, pos_b, size = get_overlap(query_str, word)
        overlap = query_str[pos_a:pos_a+size]
#         print(overlap)
        overlap_indices = path[pos_a:pos_a+size]
        return len(overlap) / len(query_str), sum(indices_logits[overlap_indices])
    return 0

In [16]:
# encode entity label
query_str = s_label
# query_str = 'gregor'
print(query_str)

q_words = [w for w in query_str.split() if len(w) > 1]
print(q_words)

acc_matches, acc_logits = 0, 0
for word in q_words:
    query_str = word.upper()
    matches, logits = match(edges, indices, query_str, tokenizer)
    acc_matches += matches
    acc_logits += logits
    
print('%.2f words matched with mass %.2f' % (acc_matches/len(q_words), acc_logits))

wiebke carolsfeld
['wiebke', 'carolsfeld']
1.00 words matched with mass 116.41


In [18]:
# search through all pre-selected entity labels
for query_str in top_labels:
#     print(query_str)

    q_words = [w for w in query_str.split() if len(w) > 1]
    
    acc_matches, acc_logits = 0, 0
    for word in q_words:
        query_str = word.upper()
        matches, logits = match(edges, indices, query_str, tokenizer)
        acc_matches += matches
        acc_logits += logits
    if acc_matches/len(q_words) == 1 and acc_logits > 70:
        print(q_words)
        print('%.2f words matched with mass %.2f' % (acc_matches/len(q_words), acc_logits))
#     break

['wiebke', 'carolsfeld']
1.00 words matched with mass 116.41
['film', 'director']
1.00 words matched with mass 77.69
